In [25]:
import json

gender_map = {
    "m sg": "陽性單數",
    "f sg": "陰性單數",
    "n sg": "中性單數",
    "m pl": "陽性複數",
    "f pl": "陰性複數",
    "n pl": "中性複數",
    "pl": "複數"
}

def convert_gender_number(gn):
    """將 'm sg' 等字串轉換為中文語意。若無對應則原樣返回"""
    return gender_map.get(gn, gn)

def convert_case(case):
    """將德文格位轉為中文描述"""
    case_map = {
        "Akkusativ": "第四格",
        "Dativ": "第三格",
        "Genitiv": "第二格"
    }
    return case_map.get(case, case)

def json_to_md(json_file, output_file):
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    md_lines = []
    md_lines.append("# **動詞整理**\n\n")

    for vp in data["verb_phrases"]:
        lemma = vp["lemma"]
        
        # 處理多層次翻譯
        zh_meanings = vp.get("zh_meanings", {})
        original_meanings = zh_meanings.get("original", [])
        contextual_meaning = zh_meanings.get("contextual", "")
        
        # 標題：動詞原形
        md_lines.append(f"## {lemma}\n")
        
        # 中文含義（多層次）
        if original_meanings or contextual_meaning:
            md_lines.append("- **中文含義**：\n")
            if original_meanings:
                md_lines.append(f"  - 原意：{', '.join(original_meanings)}\n")
            if contextual_meaning:
                md_lines.append(f"  - 語境意義：{contextual_meaning}\n")

        # grammar 區塊
        grammar_info = vp.get("grammar", {})
        case_req = grammar_info.get("case_requirement")
        is_reflexive = grammar_info.get("is_reflexive", False)
        is_modal = grammar_info.get("is_modal", False)
        gov_prep = grammar_info.get("gov_preposition")
        is_separable = grammar_info.get("is_separable_verb", False)

        if case_req:
            md_lines.append(f"- **受詞格位需求**：{convert_case(case_req)}\n")
        if is_reflexive:
            md_lines.append("- **特性**：反身動詞\n")
        if is_modal:
            md_lines.append("- **特性**：情態動詞\n")
        if is_separable:
            md_lines.append("- **特性**：分離動詞\n")
        if gov_prep:
            md_lines.append(f"- **主要支配介詞**：{gov_prep}\n")
        
        md_lines.append("\n")

        # 範例句分析
        examples = vp.get("examples", [])
        if examples:
            md_lines.append("### 例句分析\n")
        
        for idx, ex in enumerate(examples, 1):
            md_lines.append(f"{idx}. **德文例句**：`{ex['de']}`\n")
            md_lines.append(f"   - 中文翻譯：{ex['zh']}\n")

            comp = ex.get("components", {})
            
            # 主語
            subject_data = comp.get("subject")
            if subject_data:
                subj_de = subject_data.get("de", "")
                subj_zh = subject_data.get("zh", "")
                subj_gn = subject_data.get("gender_number", "")
                subj_gn_zh = convert_gender_number(subj_gn) if subj_gn else ""
                subj_orig = subject_data.get("original_nominative_form", "")

                md_lines.append(f"   - 主語：{subj_de}（{subj_zh}）\n")
                if subj_gn_zh:
                    md_lines.append(f"       * 性質：{subj_gn_zh}\n")
                if subj_orig and subj_de != subj_orig:
                    md_lines.append(f"       * 原形：{subj_orig}\n")

            # 動詞形態
            verb_form = comp.get("verb_form", "")
            md_lines.append(f"   - 動詞形態：{verb_form}\n")

            # 受詞 objects
            objects = comp.get("objects", [])
            for obj in objects:
                obj_de = obj.get("de", "")
                obj_zh = obj.get("zh", "")
                obj_case = obj.get("case", "")
                obj_gn = obj.get("gender_number", "")
                obj_gn_zh = convert_gender_number(obj_gn) if obj_gn else ""
                obj_orig = obj.get("original_nominative_form", "")
                
                md_lines.append(f"   - 受詞：{obj_de}（{obj_zh}, {convert_case(obj_case)}）\n")
                if obj_gn_zh:
                    md_lines.append(f"       * 性質：{obj_gn_zh}\n")
                if obj_orig:
                    md_lines.append(f"       * 原形：{obj_orig}\n")

            # 不同語法元素之間空一行
            md_lines.append("")

        # 分隔線
        md_lines.append("\n***\n")

    # 元數據輸出
    metadata = data.get("metadata", {})
    md_lines.append(f"> **資料來源**：{metadata.get('source', '不明')}  \n")
    md_lines.append(f"> **語言版本**：{metadata.get('language', '不明')}  \n")
    md_lines.append(f"> **處理時間**：{metadata.get('processed_date', '不明')}  \n")
    md_lines.append(f"> **版本**：{metadata.get('version', '未知')}\n")

    # 寫入 Markdown 檔案
    with open(output_file, 'w', encoding='utf-8') as f:
        f.writelines(md_lines)

In [26]:
 

# 使用範例
# json_to_md('german_verbs_tw.json', 'output_verbs_tw.md')


# 使用範例
json_to_md('german_verbs_tw_02.json', 'output_verbs_tw_02.md')